# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `datess.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices. **This Series should be sorted by a tie-break sort in the format of ("extracted date", "original row number").**

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [3]:
import pandas as pd


doc = []
with open('assets/dates.txt') as file:
    for line in file:
        doc.append(line)
df = pd.Series(doc)



In [68]:
def date_sorter():
    import re
    from datetime import datetime
    from calendar import month_name
    import dateutil.parser
    df = pd.DataFrame(doc, columns=['text'])
    df['text'] = df['text'].apply(lambda x: x.strip('\n'))
    pattern = "[,.]? \d{4}|".join(month_name[1:]) + "[,.]? \d{4}"
  
    # data variants in the order of likelihood 
    date_formats = [
        r'\d{1,2}[/-]\d{1,2}[/-]\d{4}',
        r'\d{1,2}/\d{1,2}/\d{2}',
        r'\d{1,2}-\d{1,2}-\d{2}',
        r'\d{1,2}\s(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*.?\s\d{4}',
        r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*.?\s\d{2},?\s\d{4}',
        r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*,?\s\d{4}'
        r'\d{1,2}\/\d{1,2}\/\d{2,4}|\d{1,2}\-\d{1,2}\-\d{2,4}|[A-Z][a-z]+\-\d{1,2}\-\d{4}|[A-Z][a-z]+[,.]? \d{2}[a-z]*,? \d{4}|\d{1,2} [A-Z][a-z,.]+ \d{4}|[A-Z][a-z]{2}[,.]? \d{4}|'+pattern+r'|\d{1,2}\/\d{4}|\d{4}',
        r'\d{1,2}/\d{4}',
        r'\d{1,2}-\d{4}'
        r'[1,2]\d{3}'
    ]
    dates = []
    for line in df['text']:
        for date_format in date_formats:
            matches = re.findall(date_format, line)
            dates.extend(matches)
    dates = [date for date in dates if date != []]
    dates_list = list(dates)
    parsed_dates = []
    for year in dates_list:
        try:
            if re.match(r'\d{4}', year):
                parsed_date = datetime.strptime('January 1, ' + year, "%B %d, %Y")
            elif re.match(r'\d{1,2}\/\d{4}', year):
                date_split = year.split('/')
                parsed_date = datetime.strptime(f'{date_split[0]}/1/{date_split[1]}', "%m/%d/%Y")
            elif re.match(r'[A-Z][a-z]+[,.]? \d{4}', year):
                date_split = year.split(' ')
                parsed_date = datetime.strptime(f'{date_split[0]} 1 {date_split[1]}', "%B %d, %Y")
            else:
                parsed_date = dateutil.parser.parse(year)


            parsed_dates.append(parsed_date)
        except (ValueError, OverflowError):
            continue
    df['parsed_dates'] = pd.Series(parsed_dates)
    df = df.sort_values(by='parsed_dates')
    df = df.reset_index()
    return pd.Series(df['index'])

   
            
        
date_sorter()

0      336
1      326
2      343
3      296
4      301
      ... 
495    495
496    496
497    497
498    498
499    499
Name: index, Length: 500, dtype: int64

In [6]:


def date_sorter():
    import re
    from datetime import datetime
    from calendar import month_name
    import dateutil.parser
    df = pd.DataFrame(doc, columns=['text'])
    df['text'] = df['text'].apply(lambda x: x.strip('\n'))

    pattern = "[,.]? \d{4}|".join(month_name[1:]) + "[,.]? \d{4}"

    date_formats = [
        r'\d{1,2}[/-]\d{1,2}[/-]\d{4}',
        r'\d{1,2}/\d{1,2}/\d{2}',
        r'\d{1,2}-\d{1,2}-\d{2,4}',
        r'\d{1,2}\s(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*.?\s\d{4}',
        r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*.?\s\d{2},?\s\d{4}',
        r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*,?\s\d{4}'
        r'\d{1,2}\/\d{1,2}\/\d{2,4}|\d{1,2}\-\d{1,2}\-\d{2,4}|[A-Z][a-z]+\-\d{1,2}\-\d{4}|[A-Z][a-z]+[,.]? \d{2}[a-z]*,? \d{4}|\d{1,2} [A-Z][a-z,.]+ \d{4}|[A-Z][a-z]{2}[,.]? \d{4}|'+pattern+r'|\d{1,2}\/\d{4}|\d{4}',
        r'\d{1,2}/\d{4}',
        r'\d{1,2}-\d{4}'
        r'[1,2]\d{3}'
    ]

    dates = []
    for line in df['text']:
        for date_format in date_formats:
            matches = re.findall(date_format, line)
            dates.extend(matches)

    dates = [date for date in dates if date != []]
    dates_list = list(dates)

    parsed_dates = []
    for year in dates_list:
        try:
            if re.match(r'\d{4}', year):
                parsed_date = datetime.strptime('January 1, ' + year, "%B %d, %Y")
            elif re.match(r'\d{1,2}\/\d{4}', year):
                date_split = year.split('/')
                parsed_date = datetime.strptime(f'{date_split[0]}/1/{date_split[1]}', "%m/%d/%Y")
            elif re.match(r'[A-Z][a-z]+[,.]? \d{4}', year):
                date_split = year.split(' ')
                parsed_date = datetime.strptime(f'{date_split[0]} 1 {date_split[1]}', "%B %d, %Y")
            else:
                parsed_date = dateutil.parser.parse(year)

            if parsed_date.year < 1900 or parsed_date.year > 2022:
                raise ValueError("Invalid year")

            parsed_dates.append(parsed_date)
        except (ValueError, OverflowError):
            continue

    df['parsed_dates'] = pd.Series(parsed_dates)
    df = df.sort_values(by='parsed_dates')
    df = df.reset_index()
    return pd.Series(df['index'])

# Call the function
date_sorter()


0      222
1      212
2      229
3      182
4      187
      ... 
495    495
496    496
497    497
498    498
499    499
Name: index, Length: 500, dtype: int64

0      474
1      153
2      129
3      225
4      171
      ... 
495     98
496    111
497     31
498     36
499     57
Length: 500, dtype: int64